# Feature engineering with Pandas


## Импорты


In [ ]:
import yaml

with open("../../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score

# from xgboost import XGBRegressor

### Общая информация


In [ ]:
train_df = pd.read_csv(cfg["house_pricing"]["train_dataset"])
train_df.head()

In [ ]:
test_df = pd.read_csv(cfg["house_pricing"]["test_dataset"])
test_df.head()

Не все столбцы здесь выведены. Их список мы можем получить, используя аттрибут `columns`:


In [ ]:
train_df.columns

Почистим данные в нескольких столбцах, основываясь на data_description


In [ ]:
train_df["Exterior2nd"] = train_df["Exterior2nd"].replace({"Brk Cmn": "BrkComm"})
# Some values of GarageYrBlt are corrupt, so we'll replace them
# with the year the house was built
train_df["GarageYrBlt"] = train_df["GarageYrBlt"].where(
    train_df.GarageYrBlt <= 2010, train_df.YearBuilt
)
# Names beginning with numbers are awkward to work with
train_df.rename(
    columns={
        "1stFlrSF": "FirstFlrSF",
        "2ndFlrSF": "SecondFlrSF",
        "3SsnPorch": "Threeseasonporch",
    },
    inplace=True,
)

In [ ]:
cat_df = train_df.select_dtypes(include=["object"])

In [ ]:
num_df = train_df.select_dtypes(exclude=["object"])

## Коррелирующие признаки


In [ ]:
fig, axes = plt.subplots(8, 5, figsize=(20, 20))
axes_flattened = axes.reshape(-1)
for i in range(len(num_df.columns)):
    ax = axes_flattened[i]
    sns.scatterplot(
        x=num_df.iloc[:, i],
        y="SalePrice",
        data=num_df.dropna(),
        ax=ax,
    )
fig.tight_layout(pad=1.0)

In [ ]:
def corrplot(df, method="pearson", annot=True, **kwargs):
    sns.clustermap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )


corrplot(num_df, annot=None)

In [ ]:
def corrplot(df, method="pearson", annot=True, **kwargs):

    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        annot=annot,
        ax=ax,
        **kwargs,
    )


corrplot(num_df, annot=None)

Из этой матрицы можно увидеть, какие столбцы сильно коррелируют между собой, например:

1. GarageYrBlt и YearBuilt
2. TotRmsAbvGrd и GrLivArea
3. FirstFlrSF и TotalBsmtSF
4. GarageArea и GarageCars


**Задание**: Выведите 5 пар наиболее сильно коррелирующих признаков. Сколько пар признаков коррелирует больше, чем на 0.75?


In [ ]:
m = num_df.corr().abs()
np.fill_diagonal(m.values, 0)
pairs = m.stack().reset_index()
pairs.columns = ["1", "2", "corr"]
pairs = pairs.sort_values(by="corr", ascending=False)
print(pairs.head(5))
print((pairs["corr"] > 0.75).sum())

**Задание**: Постройте карту корреляций, но не используя clustermap. Используйте для этого heatmap. Ответьте на вопрос, те же пары признаков наиболее ярко подсвечены или разные? По какому из графиков вам удобнее делать выводы?


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(
    num_df.corr("pearson"), vmin=-1.0, vmax=1.0, cmap="icefire", annot=None, ax=ax
)

**Задание**: Выбросьте несколько лишних признаков из датасета.


In [ ]:
train_df.drop(
    ["GarageYrBlt", "TotRmsAbvGrd", "FirstFlrSF", "GarageCars"], axis=1, inplace=True
)

## Заполнение пустых значений


Может быть множество вариантов, при которых строка может содержать пустые значения. Например:

1. Дом с 2 спальнями не может включать ответ на вопрос, насколько велика третья спальня
2. Кто-то из опрошенных может не делиться своим доходом
3. ...

Библиотеки Python представляют недостающие числа как NaN-ми, что является сокращением от "not a number".

Соберем статистику, связанную с NaN-ми. Какие ячейки имеют недостающие значения (в процентах), можно определить с помощью команды:


In [ ]:
nan_df = (train_df.isnull().mean() * 100).reset_index()
nan_df

Преобразуем этот датафрейм в более изящный вид:


In [ ]:
nan_df.columns = ["column_name", "percentage"]
nan_df.sort_values("percentage", ascending=False, inplace=True)
nan_df.head(10)

In [ ]:
test_df.columns

Выведем квантили:


In [ ]:
intro_srt = "Number of columns with more than"
for percent in (80, 50, 20, 5):
    print(f"{intro_srt} {percent}% NANs: {(nan_df.percentage > percent).sum()}")

Выведем столбцы с более чем 80% NaN-в


In [ ]:
nan_columns = list(nan_df[nan_df.percentage > 80]["column_name"])
nan_columns

Большинство моделей не умеют работать с NaN-ми. Поэтому требуется избавиться от них.
Вообще есть три вида отсутствия значений: Полностью случайное, случайное и неслучайное.

-   Полностью случайное не связано ни с какими другими зависимостями в датасете. Удаление таких данных никак не повлияет на результаты.
-   Случайное связано с наблюдаемыми переменными, но не с ненаблюдаемыми. Это самый широкий класс, и с ним борется большинство методов.
-   Случайное, которое связано с ненаблюдаемыми переменными (например, старением оборудования).
    В зависимости от того, с каким видом пропусков мы имеем дело, можно использовать разные методы.

### Выброс стоблцов с NaN-ми


In [ ]:
# 1 вариант - если, например, нужно выбросить одинаковые
# столбцы для обучающей и тестовой выборок
num_сols_with_missing = [col for col in num_df.columns if num_df[col].isnull().any()]
num_сols_with_missing

In [ ]:
print(len(num_df.columns))
num_df_dropped = num_df.drop(num_сols_with_missing, axis=1)
print(len(num_df_dropped.columns))

**2 опция:** выбросить столбцы, напрямую используя `dropna()`


In [ ]:
print(len(num_df.columns))
num_df_dropped = num_df.dropna(axis=1)
print(len(num_df_dropped.columns))

Если эти столбцы содержат полезную информацию (в местах, которые не были пропущены), модель теряет доступ к этой информации при удалении столбца. Кроме того, если тестовые данные имеют отсутствующие значения в тех местах, где тренировочные не имели, это приведет к ошибке.
Однако оно может быть полезно, когда большинство значений в столбце отсутствуют.


**Задание**: выбросьте столбцы, у которых отсутствует больше 75% значений. Сделайе это и для тренировочных, и для тестовых данных


In [ ]:
def remove_columns(data):
    missing_percentage = data.isnull().mean()
    columns_to_drop = missing_percentage[missing_percentage > 0.75].index
    data.drop(columns=columns_to_drop, inplace=True)
    return data


train_df = remove_columns(train_df)
test_df = remove_columns(test_df)

print("train", train_df.shape)
print("test", test_df.shape)

nan_percentage_train = train_df.isnull().mean() * 100
nan_percentage_test = test_df.isnull().mean() * 100

for percent in (75, 50, 20, 5):
    print(f"{percent}% train nan {(nan_percentage_train > percent).sum()}")
    print(f"{percent}% test nan {(nan_percentage_test > percent).sum()}")

### Заполнение недостающих значений каким-то значением

Это значение будет не совсем правильным в большинстве случаев, но обычно оно дает более точные модели, чем полное удаление столбца.


**Числовые признаки**

Поведение по умолчанию заполняет столбец средним значением в заполненных ячейках. Существуют и более сложные стратегии.


In [ ]:
from sklearn.impute import SimpleImputer

# --- restore values ---
train_df = pd.read_csv(cfg["house_pricing"]["train_dataset"])
nan_df = (train_df.isnull().mean() * 100).reset_index()
nan_df.columns = ["column_name", "percentage"]
nan_df.sort_values("percentage", ascending=False, inplace=True)
nan_columns = list(nan_df[nan_df.percentage > 80]["column_name"])
print(nan_columns)
# --- restore values ---

my_imputer = SimpleImputer()

filled_cols = my_imputer.fit_transform(train_df[num_сols_with_missing])

In [ ]:
filled_cols

Альтернативно можно заполнить столбцы средним напрямую (или нулями, или чем угодно)


С точки зрения статистики такое заполнение оправдано, если все между признаками нет явной зависимости. В таком случае замена пропусков средними значениями не вносит смещения. Однако, часто условие независимости нарушается.

Взглянем на распределения средних значений по районам.


In [ ]:
neigh_grouped = train_df.groupby("Neighborhood")
neigh_lot = neigh_grouped["LotFrontage"].mean().reset_index(name="LotFrontage_mean")
neigh_garage = neigh_grouped["GarageArea"].mean().reset_index(name="GarageArea_mean")

fig, axes = plt.subplots(1, 2, figsize=(22, 8))
axes[0].tick_params(axis="x", rotation=90)
sns.barplot(x="Neighborhood", y="LotFrontage_mean", data=neigh_lot, ax=axes[0])
axes[1].tick_params(axis="x", rotation=90)
sns.barplot(x="Neighborhood", y="GarageArea_mean", data=neigh_garage, ax=axes[1])

В данном примере свойства домов сильно зависят от того, в каком районе они расположены. Поэтому средние значения лучше считать по районам. Это будет простой вариант построения модели для заполнения значений. Вы также можете строить линейные модели или любые другие.


In [ ]:
train_df["LotFrontage"] = train_df.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.mean())
)
train_df["GarageArea"] = train_df.groupby("Neighborhood")["GarageArea"].transform(
    lambda x: x.fillna(x.mean())
)

Заполним все оставшиеся числовые признаки средними (ранее мы не сохраняли результат в `train_df`)


In [ ]:
train_df[num_сols_with_missing] = train_df[num_сols_with_missing].fillna(
    train_df[num_сols_with_missing].mean()
)

Те столбцы, которые содержали более 80% NANов, удалим совсем


In [ ]:
train_df.drop(nan_columns, inplace=True, axis=1)

**Категориальные (номинальные) признаки**

Для заполнения категориапльных признаков естественно использовать значение по умолчанию "NA" или "missing".

Понятие среднего здесь тяжело использовать, поэтому проще заполнить модой, то есть наиболее часто встречающимся значением.


In [ ]:
train_df.columns

In [ ]:
cols = [
    "MasVnrType",
    "MSZoning",
    "Exterior1st",
    "Exterior2nd",
    "SaleType",
    "Electrical",
    "Functional",
]

for col in cols:
    print(f"Mode of column {col} is {train_df[col].dropna().mode()[0]}")


def fillna(x):
    mode_value = x.dropna().mode()
    if not mode_value.empty:
        return x.fillna(mode_value[0])
    else:
        return x


train_df[cols] = train_df.groupby("Neighborhood")[cols].transform(fillna)

**Порядковые признаки**

Значение "NA" удобно ассоциировать с нулем. Но такая замена меняет число уровней у признака, что может быть нежелательно. Более того, часто неочевидно, какой уровень признака соответствует NA.


In [ ]:
cat = [
    "GarageType",
    "GarageFinish",
    "BsmtFinType2",
    "BsmtExposure",
    "BsmtFinType1",
    "GarageCond",
    "GarageQual",
    "BsmtCond",
    "BsmtQual",
    "FireplaceQu",
    "KitchenQual",
    "HeatingQC",
    "ExterQual",
    "ExterCond",
]
train_df[cat] = train_df[cat].fillna("NA")

Одним из основных недостатков простого заполнения является то, что оно может привести к невозможным конфигурациям в данных. Представьте, что гараж отсутствует, но мы знаем число мест для машин. И хотя это неверно с точки зрения логики, это может быть все равно улучшением относительно ситуации, когда модель вообще не работала на примере с пропуском.


**Задание**: Используя simple imputer или заполнение модой на исходных тренировочных данных (вероятно, копии), создайте и найдите невозможную комбинацию (комбинации) в данных. Покажите пример(ы).


In [ ]:
from sklearn.impute import SimpleImputer

train_df_copy = train_df.copy()
print(train_df_copy.shape)

num_cols = train_df_copy.select_dtypes(include=["float64", "int64"]).columns
imputer = SimpleImputer(strategy="most_frequent")
train_df_copy[num_cols] = imputer.fit_transform(train_df_copy[num_cols])

cat_cols = train_df_copy.select_dtypes(include=["object"]).columns
train_df_copy[cat_cols] = train_df_copy[cat_cols].fillna("NA")

c_1 = pd.DataFrame(
    {
        "GarageArea": [0],
        "GarageCars": [2],
        "GarageType": ["NA"],
        "SalePrice": [200000],
    }
)

train_df_copy = pd.concat([train_df_copy, c_1], ignore_index=True)
print(train_df_copy.shape)

c_1 = train_df_copy[
    (train_df_copy["GarageArea"] == 0) & (train_df_copy["GarageCars"] > 0)
]

print(c_1.shape)

Другой минус константного заполнения - уменьшение дисперсии в данных, изменение распределения. Это происходит, так как мы добавляем много примеров с нулевой дисперсией. В том числе и поэтому полезно смотреть на зависмимость признака от других переменных.


**Задание**: Возьмите любой числовой столбец с достаточно хорошим распределением (в идеале, чтобы у гистограммы было несколько "горбов". Можете сгенерировать или взять из данных. Выбросьте из него 10, 20, 50 и 70% примеров. Покажите на графиках, как меняется гистограмма.


In [ ]:
data1 = np.random.normal(loc=40, scale=8, size=800)
data2 = np.random.normal(loc=80, scale=16, size=1600)
data3 = np.random.normal(loc=120, scale=8, size=400)

data = np.concatenate([data1, data2, data3])


def remove_data(data, percent):
    n = int(len(data) * percent / 100)
    i_remove = np.random.choice(len(data), n, replace=False)
    return np.delete(data, i_remove)


plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
sns.histplot(data, bins=50, kde=True)
plt.title("100")

data_20 = remove_data(data, 20)
plt.subplot(2, 2, 2)
sns.histplot(data_20, bins=50, kde=True)
plt.title("-20%")

data_50 = remove_data(data, 50)
plt.subplot(2, 2, 3)
sns.histplot(data_50, bins=50, kde=True)
plt.title("-50%")

data_70 = remove_data(data, 70)
plt.subplot(2, 2, 4)
sns.histplot(data_70, bins=50, kde=True)
plt.title("-70%")

plt.tight_layout()
plt.show()

**Задание**: Попробуйте найти зависимость какой-нибудь категориальной переменной от других (от одного столбца). Заполните ее с учетом этой зависимости.


In [ ]:
train_df_copy.columns

In [ ]:
# --- restore values ---
train_df = pd.read_csv(cfg["house_pricing"]["train_dataset"])
# --- restore values ---

train_df_copy = train_df.copy()
missing_before = train_df_copy["Fence"].isnull().sum()
garage_price_avg = train_df_copy.groupby("Fence")["SalePrice"].mean()
train_df_copy["Fence"] = train_df_copy["Fence"].fillna(train_df_copy["Fence"].mode()[0])
missing_after = train_df_copy["Fence"].isnull().sum()

plt.figure(figsize=(12, 7))

sns.stripplot(
    data=train_df_copy,
    x="Fence",
    y="SalePrice",
    palette="viridis",
    hue="Fence",
)
plt.title("SalePrice/Fence", fontsize=16, fontweight="bold")
plt.xlabel("Fence", fontsize=14)
plt.ylabel("SalePrice", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.grid(True, linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()

print(f"missing_before {missing_before}")
print(f"missing_after {missing_after}")

## Удаление признаков со слабой вариативностью

**Признаки с одним типичным значением**

Некоторые признаки в основном состоят из одного значения или нулей, что не особо полезно для нас. Поэтому мы устанавливаем пороговое значение, определяемое пользователем, на уровне 96%. Если столбец имеет более 96% от одного и того же значения, мы считаем признак бесполезными и удалим его.


In [ ]:
def get_almost_constant_columns(df, dropna=True):
    cols = []
    for i in df:
        if dropna:
            counts = df[i].dropna().value_counts()
        else:
            counts = df[i].value_counts()
        most_popular_value_count = counts.iloc[0]
        if (most_popular_value_count / len(df)) * 100 > 96:
            cols.append(i)
    return cols

In [ ]:
cat_df = train_df.select_dtypes(include=["object"])
overfit_cat = get_almost_constant_columns(cat_df)
train_df = train_df.drop(overfit_cat, axis=1)
overfit_cat

In [ ]:
num_df = train_df.select_dtypes(exclude=["object"])
overfit_num = get_almost_constant_columns(num_df, dropna=True)
train_df = train_df.drop(overfit_num, axis=1)
overfit_num

Иногда слабая вариативность все же не означает, что признак бесполезен. Это возможно для очень сильно несбалансированной переменной с малым количеством уровней. Более того, иногда такая несбалансированность возникает и в таргете!


**Признаки с маленькой дисперсией**

Другой способ - использовать метод VarianceThreshold от sklearn — это простой базовый подход к выбору признаков. Он удаляет все признаки, дисперсия которых не соответствует определенному порогу. По умолчанию он удаляет все элементы с нулевой дисперсией, т.е. те элементы, которые имеют одинаковое значение у всех семплов.

Стоит отметить, что дисперсия является абсолютной величиной, и выбор порога в этом случае является эмпирическим. При этом в общем случае малые значения дисперсии не говорят о бесполезности признака. Если признак задан на поле вещественных чисел, то его дискриминирующая способность не зависит от дисперсии, так как любой непрерывный интервал на вещественной оси содержит бесконечный набор значений. Однако, в случае дискретных значений (пример, целочисленных признаков) VarianceThreshold действительно становится полезным


In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=0.1)
num_col = train_df.select_dtypes(exclude=["object"])

sel.fit(num_col)  # fit finds the features with low variance
sum(sel.get_support())

Метод `get_support()` возвратит булевскую маску для признаков, которые проходят указанный порог по дисперсии. Ее можно использовать для отбора этих признаков


In [ ]:
sel.get_support()

Например, таким образом мы получаем список всех признаков, которые были отсеяны данным алгоритмом:


In [ ]:
num_col.columns[~sel.get_support()]

## Удаление выбросов

Удаление выбросов предотвратит воздействие экстремальных значений на производительность наших моделей.

Из скаттерплотов выше мы можем увидеть, что следующие признаки имеют экстремальные выбросы:

-   LotFrontage
-   LotArea
-   BsmtFinSF1
-   TotalBsmtSF
-   GrLivArea

Мы уберем выбросы на основе определенного порогового значения.
Эти значения мы получим из боксплотов ("ящик с усиками"):

![Boxplot](boxplot.png)


In [ ]:
out_col = ["LotFrontage", "LotArea", "BsmtFinSF1", "TotalBsmtSF", "GrLivArea"]

fig, axes = plt.subplots(1, 5, figsize=(20, 5))
for ax, col in zip(axes, out_col):
    sns.boxplot(y=train_df[col], data=train_df, ax=ax)
fig.tight_layout(pad=1.5)

In [ ]:
for col, upper_bound in (
    ("LotFrontage", 200),
    ("LotArea", 100000),
    ("BsmtFinSF1", 4000),
    ("TotalBsmtSF", 5000),
    ("GrLivArea", 4000),
):
    train_df = train_df.drop(train_df[train_df[col] > upper_bound].index)

После удаления выбросов, сильно коррелированных признаков и условных отсутствующих значений мы можем приступить к добавлению дополнительной информации для обучения нашей модели. Это делается с помощью - Feature Engineering.


**Задание**: Создайте копию данных. Удалите из числовых столбцов выбросы (используя интерквартильный размах)


In [ ]:
train_df_copy = train_df.copy()
print(train_df_copy.shape)

cols = train_df_copy.select_dtypes(include=["float64", "int64"]).columns

for col in cols:
    Q1 = train_df_copy[col].quantile(0.25)
    Q3 = train_df_copy[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    train_df_copy = train_df_copy[
        (train_df_copy[col] >= lower_bound) & (train_df_copy[col] <= upper_bound)
    ]

print(train_df_copy.shape)

**Задание**: Определите, есть ли выбросы в LotArea после группировки по районам. Есть ли районы, в которых выбросы сильнее, чем в остальных? Есть ли связь признака "наличие выброса" c качеством дома?


In [ ]:
neigh_group = train_df.groupby("Neighborhood")["LotArea"]


def out(group):
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group < lower_bound) | (group > upper_bound)]


out_neigh = neigh_group.apply(out)
out_count = out_neigh.groupby("Neighborhood").count()

plt.figure(figsize=(12, 6))
sns.barplot(x=out_count.index, y=out_count.values)
plt.title("dist/out")
plt.xlabel("dist")
plt.ylabel("out")
plt.xticks(rotation=90)
plt.show()

train_df["has_outlier"] = train_df["LotArea"].apply(
    lambda x: 1 if x in out_neigh.values else 0
)

corr = train_df[["has_outlier", "OverallQual"]].corr()
print(f"correlation {corr.loc['has_outlier', 'OverallQual']}")

In [ ]:
train_df.columns

## Feature Engineering


Feature Engineering - это техника, с помощью которой мы создаем новые признаки, которые потенциально могут помочь в прогнозировании нашей целевой переменной, которая в данном случае является SalePrice.


MSSubClass - это столбец с числовым признаком, который на самом деле можно представить как категориальный


In [ ]:
train_df["MSSubClass"].value_counts()

In [ ]:
train_df["MSSubClass"] = train_df["MSSubClass"].apply(str)

In [ ]:
ordinal_map = {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0}
fintype_map = {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0}
expose_map = {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0}
fence_map = {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0}

In [ ]:
ord_col = [
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "HeatingQC",
    "KitchenQual",
    "GarageQual",
    "GarageCond",
    "FireplaceQu",
]
for col in ord_col:
    train_df[col] = train_df[col].map(ordinal_map)

fin_col = ["BsmtFinType1", "BsmtFinType2"]
for col in fin_col:
    train_df[col] = train_df[col].map(fintype_map)

train_df["BsmtExposure"] = train_df["BsmtExposure"].map(expose_map)

Основываясь на текущих признаках, мы можем добавить первый дополнительный признак, который будет называться TotalLot и который суммирует LotFrontage и LotArea для определения общей площади земли, доступной в виде лота. Мы также можем рассчитать общее количество площади поверхности дома, TotalSF, сложив площадь от 1-го этажа и 2-го этажа. TotalBath также может быть использован, чтобы сказать нам в общей сложности, сколько ванных комнат есть в доме. Мы также можем добавить все различные типы крыльц вокруг дома и обобщить в общей площади крыльца, TotalPorch.

-   TotalLot = LotFrontage + LotArea
-   TotalSF = TotalBsmtSF + 2ndFlrSF
-   TotalBath = FullBath + HalfBath
-   TotalPorch = OpenPorchSF + EnclosedPorch + ScreenPorch
-   TotalBsmtFin = BsmtFinSF1 + BsmtFinSF2


In [ ]:
train_df["TotalLot"] = train_df["LotFrontage"] + train_df["LotArea"]
train_df["TotalBsmtFin"] = train_df["BsmtFinSF1"] + train_df["BsmtFinSF2"]
train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["2ndFlrSF"]
train_df["TotalBath"] = train_df["FullBath"] + train_df["HalfBath"]
train_df["TotalPorch"] = (
    train_df["OpenPorchSF"] + train_df["EnclosedPorch"] + train_df["ScreenPorch"]
)

Иногда очень важно применить к признаку какое-то выпуклое преобразование. Например, логарифм, корень или наоборот возведение в степень.


In [ ]:
train_df.columns

In [ ]:
train_df["LivLotRatio"] = train_df["GrLivArea"] / train_df["LotArea"]

Мы также включаем создание бинарных столбцов для некоторых признаков, которые могут указывать на наличие(1) / отсутствие(0) некоторых признаков дома


In [ ]:
cols = [
    "MasVnrArea",
    "TotalBsmtFin",
    "TotalBsmtSF",
    "2ndFlrSF",
    "WoodDeckSF",
    "TotalPorch",
]

for col in cols:
    col_name = col + "_bin"
    train_df[col_name] = train_df[col].apply(lambda train_df: 1 if train_df > 0 else 0)

Наконец, поскольку нам нужны данные, которые носят численный характер, мы преобразуем оставшиеся категориальные столбцы с помощью one-hot-encoding с помощью метода get_dummies() в числовые столбцы, которые подходят для подачи в наш алгоритм машинного обучения.


In [ ]:
train_df = pd.get_dummies(train_df)

In [ ]:
len(train_df.columns)

In [ ]:
train_df.head()

Для создания новых признаков можно использовать обучение без учителя, например, кластеризацию с помощью k средних. Можно использовать как категорию (столбец с 0, 1, 2,...) метки кластеров или расстояние наблюдений до каждого кластера. Эти особенности иногда могут быть эффективными при распутывании сложных пространственных отношений.


In [ ]:
cluster_features = [
    "LotArea",
    "TotalBsmtSF",
    "2ndFlrSF",
    "GrLivArea",
]


def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new


def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)
    X_cd = kmeans.fit_transform(X_scaled)
    # Label features and join to dataset
    X_cd = pd.DataFrame(X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
    return X_cd

In [ ]:
cluster_df = cluster_labels(train_df, features=cluster_features)

In [ ]:
cluster_df.head()

In [ ]:
train_df["cluster_num"] = cluster_df["Cluster"]

## Скейлинг


Есть две основные причины, зачем нам скейлинг:

1. Многие модели учитывают масштабы для построения прогнозов
2. Многие алгоритмы сходятся быстрее для шкалированных данных.

Часто используются логарифмирование для данных с сильно несимметричным распределением. Логарифмирование позволяет также решить проблему нелинейности в масштабе. Основная идея в переводе мультипликативной зависимости в аддитивную.
Для последнего рассмотрим следующий пример: Для концертов с различным масштабом увеличение аудитории на константу имеет разную значимость. Рассмотрим, что происходит каждый раз, когда размер аудитории увеличивается на 50%. Для начального размера аудитории 100 происходит следующее:
$$ \log*{10}(150) = \log*{10}(100 \cdot 1.5) = \log*{10}(100) + \log*{10}(1.5) \approx 2 + 0.176 $$
Для аудитории в 1000:
$$ \log*{10}(15,000) = \log*{10}(10,000 \cdot 1.5) = \log*{10}(10,000) + \log*{10}(1.5) \approx 4 + 0.176 $$
Так мы можем уменьтшить сатурацию важности признака в зависисмости от масштаба. Важно понимать, что мы не пытаемся нормализовать переменную, а именно решить проблему скошенности.

**Задание**: Найдите такой признак в наших данных и прологарифмируйте.

Что делать, если переменная может быть отрицательной? Для этого можно использовать
$$ y = log\_{base}(x + offset) $$
offset позволяет сдвинуть значения в нужную сторону, и лучше он должен быть меньше, чем минимальное возможное. (Если, конечно, оно известно).


In [ ]:
feature = "LotArea"
train_df_copy = train_df.copy()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(train_df_copy[feature], bins=50, kde=True)
plt.title(feature)

train_df_copy[f"log_{feature}"] = np.log1p(train_df_copy[feature])

plt.subplot(1, 2, 2)
sns.histplot(train_df_copy[f"log_{feature}"], bins=50, kde=True)
plt.title(f"log({feature})")

plt.show()

Это не универсальное решение. Хотя оно может улучшать распределения с перекосом вправо (где большинство точек данных имеют более низкие значения), он оказывает противоположный эффект на распределение, которое не искажено или смещено влево.


Есть ли альтернативы? Итак, нам нужен метод, который преобразует шкалу в неперекошенную, а также работает с отрицательными данными.

Квадратный корень может быть тем, что мы ищем. Сам по себе он принимает на вход положительное число и возвращает число, которое при умножении на себя равно входному значению. Это дает желаемый эффект сжатия: большие значения сжимаются сильнее, чем меньшие. Кроме того, поскольку его областью определения являются положительные числа (0 — это особый случай, поскольку он отображается сам в себя), мы можем отразить его, чтобы работать с отрицательными числами так же, как он работал с положительными числами. Это приводит к следующему преобразованию:
$$ y = \text{sign}(x)\sqrt{\left| x \right|} $$

У него нет такой возможности сжимать большие значения, как у логарифмов, но зато оно будет легко работать с отрицательными значениями и позволит уловить квадратичные эффекты. Однако, хотя оно сохраняет порядок числовых значений, но не дает нам хорошего способа интерпретировать изменения (и может исказить неискаженные распределения).


### Скейлинг с обучением

RobustScaler - это метод преобразования, который удаляет медиану и масштабирует данные в соответствии с диапазоном квантиля (по умолчанию IQR: межквартильный диапазон). IQR - это диапазон между 1-м квартилем (25-й квантилем) и 3 Квартиль (75-й квантиль). Он также устойчив к пропускам, что делает его идеальным для данных, где слишком много пропусков, что резко сокращает количество обучающих данных.

Запуская скейлер как на тренировочном, так и на тестовом наборах, мы подвергаем себя проблеме утечки данных. Утечка данных - это проблема, когда для создания модели используется информация извне набора для обучения. Если мы подгоняем скейлер как на тренировочные, так и на тестовые данные, наши характеристики тренировочных данных будут содержать распределение нашего тестового набора. Таким образом, мы неявно передаем информацию о наших тестовых данных в окончательные тренировочные данные для обучения, что не даст нам возможности по-настоящему протестировать нашу модель на данных, которые она никогда не видела.

_Извлеченные уроки:_ Установка скейлера только на обучающие данные, а затем преобразование данных как обучающей, так и тестовой выборок


In [ ]:
from sklearn.preprocessing import RobustScaler

cols = train_df.select_dtypes(np.number).columns
# train_df = train_df.drop(["Id"], axis=1)
transformer = RobustScaler().fit(train_df[cols])
train_df[cols] = transformer.transform(train_df[cols])

Таким образом, мы сильно преобразовали наш обучающий набор. Кроме перчисленного, полезно использовать PCA, выбор признаков на основе информации и други методы. Как вы, наверное, заметили, все преобразования были сделаны только для тренировочного набора, но то же самое необходимо сделать и для тестового.

Чтобы предотвратить утечку данных, все преобразования по среднему и тп нужно сделать независимо, а если мы, например, кодировали или удаляли столбцы, нужно сделать такое же преобразование, используя старые правила.

После того, как это было сделано, можно передавать данные в модель.


**Задание**: Используя признаки, построенные для обучающего набора, дополните тестовый. Сохраните резщультирующие наборы на будущее.


In [ ]:
if "Id" in train_df.columns:
    train_df = train_df.drop("Id", axis=1)

cols = train_df.select_dtypes(include=[np.number]).columns.drop(
    "SalePrice", errors="ignore"
)

print("src")
print(f"train_df.shape: {train_df.shape}")
print(f"test_df.shape: {test_df.shape}")
print(f"cols: {list(cols)}")

scaler = RobustScaler().fit(train_df[cols])
train_df[cols] = scaler.transform(train_df[cols])

if "Id" in test_df.columns:
    test_ids = test_df["Id"]
    test_df = test_df.drop("Id", axis=1)

test_df = pd.get_dummies(test_df)
missing_cols = set(train_df.columns) - set(test_df.columns)

for col in missing_cols:
    test_df[col] = 0

test_df = test_df[train_df.columns.drop("SalePrice", errors="ignore")]
test_df[cols] = test_df[cols].fillna(0)
test_df[cols] = scaler.transform(test_df[cols])

print("\nafter")
print(f"train_df.shape: {train_df.shape}")
print(f"test_df.shape: {test_df.shape}")
print(f"{len(missing_cols)} missing_cols")

if missing_cols:
    print("cols:", list(missing_cols))

# train_df.to_csv("train_preprocessed.csv", index=False)
# test_df.to_csv("test_preprocessed.csv", index=False)

In [ ]:
train_df.head()